In [2]:
import requests
from bs4 import BeautifulSoup


class WebsiteImageParser:
    def __init__(
        self,
        url,
        save_path="test.png",
    ) -> None:
        self.url = url
        self.save_path = save_path

    def parse(self):
        """
        template method - он определяет общую структуру всего алгоритма
        """
        html = self.get_html(url=self.url)
        soup = self.get_soup(html=html)
        img_path = self.extract_img(soup=soup)
        print(f"LOG: parse image path {img_path}")
        self.save_image(img_path=img_path)
        print(f"Saved successfuly on path: {self.save_path}")

    def extract_img(self, soup):
        raise "You should write this method"

    def get_html(self, url):
        response = requests.get(url=url)
        return response.text

    def get_soup(self, html):
        soup = BeautifulSoup(html)
        return soup

    def save_image(self, img_path):
        r = requests.get(img_path, allow_redirects=True)
        with open(self.save_path, "wb") as f:
            f.write(r.content)


class WikiParser(WebsiteImageParser):
    def extract_img(self, soup):
        image = soup.find("img", class_="mw-logo-icon")
        image = image["src"]
        image = f"https://en.wikipedia.org{image}"
        return image


class GoogleParser(WebsiteImageParser):
    def extract_img(self, soup):
        image = soup.find("img")
        image = image["src"]
        image = f"https://www.google.com{image}"
        return image


wiki_parser = WikiParser(
    url="https://en.wikipedia.org/wiki/Main_Page",
    save_path="./wiki.png",
)
wiki_parser.parse()

google_parser = GoogleParser(
    url="https://www.google.com/",
    save_path="./google.png",
)

google_parser.parse()

LOG: parse image path https://en.wikipedia.org/static/images/icons/wikipedia.png
Saved successfuly on path: ./wiki.png
LOG: parse image path https://www.google.com/images/branding/googlelogo/1x/googlelogo_white_background_color_272x92dp.png
Saved successfuly on path: ./google.png


In [3]:
from abc import ABC, abstractmethod


class ParserFactory(ABC):
    @abstractmethod
    def create_parser(self, url, save_path):
        pass


class WikiParserFactory(ParserFactory):
    def create_parser(self, url, save_path):
        return WikiParser(url=url, save_path=save_path)

    def some_logic_1(self):
        print("do some logic 1")


class GoogleParserFactory(ParserFactory):
    def create_parser(self, url, save_path):
        return GoogleParser(url=url, save_path=save_path)

    def some_logic_2(self):
        print("do some logic 2")


wiki_factory = WikiParserFactory()
google_factory = GoogleParserFactory()

google_params = [
    ["https://www.google.com/", "./google_1.png"],
    ["https://www.google.com/", "./google_2.png"],
    ["https://www.google.com/", "./google_3.png"],
]

parsers: list[GoogleParser] = []

for param in google_params:
    url, save_path = param
    parser = google_factory.create_parser(
        url=url,
        save_path=save_path
    )
    parsers.append(parser)

for parser in parsers:
    parser.parse()

LOG: parse image path https://www.google.com/images/branding/googlelogo/1x/googlelogo_white_background_color_272x92dp.png
Saved successfuly on path: ./google_1.png
LOG: parse image path https://www.google.com/images/branding/googlelogo/1x/googlelogo_white_background_color_272x92dp.png
Saved successfuly on path: ./google_2.png
LOG: parse image path https://www.google.com/images/branding/googlelogo/1x/googlelogo_white_background_color_272x92dp.png
Saved successfuly on path: ./google_3.png
